In [10]:
!pip install timm
!pip install matplotlib

You should consider upgrading via the '/home/amor/Documents/code_dw/neural-styles/venv2/bin/python3 -m pip install --upgrade pip' command.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 5.6 MB/s eta 0:00:00m eta 0:00:010:00:01
  Using cached cycler-0.11.0-py3-none-any.whl (6.4 kB)
  Using cached kiwisolver-1.4.4-cp39-cp39-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (1.6 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 965.4/965.4 KB 3.7 MB/s eta 0:00:00 MB/s eta 0:00:01:01
  Using cached pyparsing-3.0.9-py3-none-any.whl (98 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.7/299.7 KB 3.7 MB/s eta 0:00:00 MB/s eta 0:00:01:01
You should consider upgrading via the '/home/amor/Documents/code_dw/neural-styles/venv2/bin/python3 -m pip install --upgrade pip' command.


In [8]:
!git clone https://github.com/Alibaba-MIIL/ImageNet21K
%cd ImageNet21K

Cloning into 'ImageNet21K'...
remote: Enumerating objects: 248, done.
remote: Counting objects: 100% (93/93), done.
remote: Compressing objects: 100% (44/44), done.
remote: Total 248 (delta 68), reused 49 (delta 49), pack-reused 155
Receiving objects: 100% (248/248), 1.40 MiB | 4.32 MiB/s, done.
Resolving deltas: 100% (130/130), done.
/home/amor/Documents/code_dw/neural-styles/notebooks/ImageNet21K


In [ ]:
# --------------------------------------------------------
# ImageNet-21K Pretraining for The Masses
# Copyright 2021 Alibaba MIIL (c)
# Licensed under MIT License [see the LICENSE file for details]
# Written by Tal Ridnik
# --------------------------------------------------------

import os
import urllib
from argparse import Namespace
import torch
from PIL import Image
from timm.data import resolve_data_config
from timm.data.transforms_factory import create_transform
from src_files.semantic.semantics import ImageNet21kSemanticSoftmax
import timm

############### Downloading metadata ##############
print("downloading metadata...")
url, filename = (
    "https://miil-public-eu.oss-eu-central-1.aliyuncs.com/model-zoo/ImageNet_21K_P/resources/fall11/imagenet21k_miil_tree.pth",
    "imagenet21k_miil_tree.pth")
if not os.path.isfile(filename):
    urllib.request.urlretrieve(url, filename)
args = Namespace()
args.tree_path = filename
semantic_softmax_processor = ImageNet21kSemanticSoftmax(args)
print("done")

############### Loading (ViT) model from timm package ##############
print("initilizing model...")
model = timm.create_model('vit_base_patch16_224_miil_in21k', pretrained=True)
model.eval()
config = resolve_data_config({}, model=model)
transform = create_transform(**config)
print("done")


In [15]:

############## Loading sample image ##############
print("downloading sample image...")
url, filename = ("https://github.com/pytorch/hub/raw/master/images/dog.jpg", "dog.jpg")
if not os.path.isfile(filename):
    urllib.request.urlretrieve(url, filename)
img = Image.open(filename).convert('RGB')
tensor = transform(img).unsqueeze(0)  # transform and add batch dimension
print("done")

############## Doing semantic inference ##############
print("doing semantic infernce...")
labels = []
with torch.no_grad():
    logits = model(tensor)
    semantic_logit_list = semantic_softmax_processor.split_logits_to_semantic_logits(logits)

    # scanning hirarchy_level_list
    for i in range(len(semantic_logit_list)):
        logits_i = semantic_logit_list[i]

        # generate probs
        probabilities = torch.nn.functional.softmax(logits_i[0], dim=0)
        top1_prob, top1_id = torch.topk(probabilities, 1)

        if top1_prob > 0.5:
            top_class_number = semantic_softmax_processor.hierarchy_indices_list[i][top1_id[0]]
            top_class_name = semantic_softmax_processor.tree['class_list'][top_class_number]
            top_class_description = semantic_softmax_processor.tree['class_description'][top_class_name]
            labels.append(top_class_description)

print("labels found {}.".format(labels))

############## Visualization ##############
import matplotlib
import os
import numpy as np

if os.name == 'nt':
    matplotlib.use('TkAgg')
else:
    matplotlib.use('Agg')
import matplotlib.pyplot as plt


plt.imshow(img)
plt.axis('off')
plt.title('Semantic labels found: \n {}'.format(np.array(labels)))
plt.savefig("test.png")

downloading sample image...
done
doing semantic infernce...
labels found ['animal', 'domestic_animal', 'dog', 'spitz', 'Samoyed'].


In [13]:
plt.get_figure()

AttributeError: module 'matplotlib.pyplot' has no attribute 'get_figure'

In [3]:
import timm
model = timm.create_model('vit_base_patch16_224_miil_in21k', pretrained=True)
model.eval()

Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-tresnet/vit_base_patch16_224_in21k_miil-887286df.pth" to /home/amor/.cache/torch/hub/checkpoints/vit_base_patch16_224_in21k_miil-887286df.pth


VisionTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
    (norm): Identity()
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (norm_pre): Identity()
  (blocks): Sequential(
    (0): Block(
      (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (attn): Attention(
        (qkv): Linear(in_features=768, out_features=2304, bias=False)
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=768, out_features=768, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (ls1): Identity()
      (drop_path1): Identity()
      (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=768, out_features=3072, bias=True)
        (act): GELU(approximate='none')
        (drop1): Dropout(p=0.0, inplace=False)
        (fc2): Linear(in_features=3072, out_features=768, bias=True)
        (drop2): Dropout(p=0.0, inplace=

In [6]:
model = timm.create_model('tresnet_m_miil_in21k', pretrained=True)
model.eval()
model

Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-tresnet/tresnet_m_miil_in21k-901b6ed4.pth" to /home/amor/.cache/torch/hub/checkpoints/tresnet_m_miil_in21k-901b6ed4.pth


TResNet(
  (body): Sequential(
    (SpaceToDepth): SpaceToDepthModule()
    (conv1): Sequential(
      (0): Conv2d(48, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (1): InplaceAbn()
    )
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Sequential(
          (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (1): InplaceAbn()
        )
        (conv2): Sequential(
          (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (1): InplaceAbn()
        )
        (relu): ReLU(inplace=True)
        (se): SEModule(
          (fc1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
          (bn): Identity()
          (act): ReLU(inplace=True)
          (fc2): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
          (gate): Sigmoid()
        )
      )
      (1): BasicBlock(
        (conv1): Sequential(
          (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 

In [7]:
dir(model)

['T_destination',
 '__annotations__',
 '__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_apply',
 '_backward_hooks',
 '_buffers',
 '_call_impl',
 '_forward_hooks',
 '_forward_pre_hooks',
 '_get_backward_hooks',
 '_get_name',
 '_is_full_backward_hook',
 '_load_from_state_dict',
 '_load_state_dict_post_hooks',
 '_load_state_dict_pre_hooks',
 '_make_layer',
 '_maybe_warn_non_full_backward_hook',
 '_modules',
 '_named_members',
 '_non_persistent_buffers_set',
 '_parameters',
 '_register_load_state_dict_pre_hook',
 '_register_state_dict_hook',
 '_replicate_for_data_parallel',
 '_save_to_state_dict',
 '_slow_forward',
 '_state_dic